# Assignment 2 - Formation Energy Regression
## Student: Marcos Gil

In [18]:
#Importing Stuff
import numpy as np
import pandas as pd
from sklearn import linear_model
import sklearn.metrics as metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV

In this assignment we intend to assess the predictive capability of different regression models. We then introduce this function that outputs some interesting metrics:

In [19]:
def evaluate_model(model,x,y):
    y_pred = model.predict(x)
    print("R2 Score: %1.3g" % model.score(x, y))
    print("Mean Absolute Error normalized by the mean absolute value of y: %1.3g" % (metrics.mean_absolute_error(y_pred,y)/np.mean(np.abs(y))))
    print("Mean Squared Error: %1.3g" % (metrics.mean_squared_error(y_pred,y)))

Here we load the dataset as a Pandas' data frame and perform some manipulations. First, we remove the first column, that simply enumerates the data. Then, we transform two categorical columns (`Material` and `Space Group`) to numbers, by simply enumerating different categories. We also fill with `0` some missing entries. Finally, we standardize all columns in order to have zero mean and unit variance. This will be important for the Lasso and Ridge regressions, which penalize "large" weights. For large to have a meaning, we have to be able to compare different features, and that can only be done if they share the same order of magnitude, which is what is achieved by this standardization.

In [20]:
df = pd.read_csv('AB2_formation_energy_materials_spacegroup.csv') #Loads

df.drop('Unnamed: 0', axis='columns', inplace=True) #Drops first column

#Converts categories (Material and Space Group) to numbers
df["Material"] = df["Material"].astype('category')
df["Space Group"] = df["Space Group"].astype('category')
df["Material"] = df["Material"].cat.codes
df["Space Group"] = df["Space Group"].cat.codes

#The compunds containing hydrogen contain nan values for the r_p_orbital_y and r_d_orbital_y columns
#We substitute these values by 0:
df.fillna(0, inplace=True)

standard_df = (df - df.mean()) / df.std()
standard_df

,Material,Space Group,hform,Z_x,Electronegativity_x,IonizationPotential_x,ElectronAffinity_x,HOMO_x,LUMO_x,r_s_orbital_x,...,r_p_orbital_y,r_d_orbital_y,r_atomic_nonbonded_y,r_valence_lastorbital_y,r_covalent_y,Valence_y,PeriodicColumn_y,PeriodicColumn_upto18_y,NumberUnfilledOrbitals_y,Polarizability_y
0,-1.827656,-0.038621,0.448420,-0.238486,0.234562,-0.122332,-0.721388,-0.144453,0.361242,-0.098198,...,0.033861,-0.933253,0.180376,0.004601,0.304477,0.948275,0.685722,0.40336,-0.999382,-0.149218
1,-1.725961,-1.188060,-0.121939,-1.813813,-0.528863,1.394351,0.420430,1.408821,1.616003,-0.840278,...,0.033861,-0.933253,0.180376,0.004601,0.304477,0.948275,0.685722,0.40336,-0.999382,-0.149218
2,-1.703363,1.110818,0.717588,-0.887150,0.830989,-1.045945,-1.009700,-0.874314,2.246358,-1.679255,...,0.033861,-0.933253,0.180376,0.004601,0.304477,0.948275,0.685722,0.40336,-0.999382,-0.149218
3,-1.703363,-1.188060,0.856338,-0.887150,0.830989,-1.045945,-1.009700,-0.874314,2.246358,-1.679255,...,0.033861,-0.933253,0.180376,0.004601,0.304477,0.948275,0.685722,0.40336,-0.999382,-0.149218
4,-1.635567,-0.038621,0.646445,1.244175,1.689842,-1.253022,-1.630741,-1.385914,0.207664,-0.424673,...,0.033861,-0.933253,0.180376,0.004601,0.304477,0.948275,0.685722,0.40336,-0.999382,-0.149218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,1.324862,1.110818,0.817949,-1.350482,-0.481149,0.285467,0.194859,-0.210620,-0.125275,0.495669,...,1.285245,-0.490119,1.201374,1.443727,0.998997,0.568600,-0.988404,-0.25832,2.376909,1.095984
621,1.415257,1.110818,1.091623,1.012509,1.260415,-0.854970,0.006985,-0.221832,-0.415809,0.183037,...,1.285245,-0.490119,1.201374,1.443727,0.998997,0.568600,-0.988404,-0.25832,2.376909,1.095984
622,1.415257,-1.188060,1.055877,1.012509,1.260415,-0.854970,0.006985,-0.221832,-0.415809,0.183037,...,1.285245,-0.490119,1.201374,1.443727,0.998997,0.568600,-0.988404,-0.25832,2.376909,1.095984
623,1.686441,1.110818,0.311491,-0.562818,-1.196860,0.732760,0.378457,0.590738,-0.509951,1.067591,...,1.285245,-0.490119,1.201374,1.443727,0.998997,0.568600,-0.988404,-0.25832,2.376909,1.095984


Now we transform the data frame to numpy arrays, separating the target `y` and the features `x`.

In [21]:
y = standard_df["hform"].to_numpy()
x = standard_df.drop('hform', axis='columns').to_numpy()

Now, we perform dimensionality reduction through Principal Component Analysis (PCA). We see that we are able to almost half the dimension of the feature space while still keeping more than 99% of the variance. We also renormalize our data and perform the train/test split.

In [93]:
pca = PCA(n_components=20) #Starting the PCA
pca.fit(x); #Fitting the measurements
print("Normalized Variance: %1.3g" % sum(pca.explained_variance_ratio_))
_reduced_x = pca.fit_transform(x)
reduced_x = (_reduced_x - np.mean(_reduced_x, axis=0)) / np.std(_reduced_x, axis=0)

xtrain,xtest,ytrain,ytest = train_test_split(reduced_x,y)

Normalized Variance: 0.993


### Direct Linear Regression

Here, we perform a standard linear regression in order to serve as a benchmark.

In [94]:
least_sqr = linear_model.LinearRegression()
least_sqr.fit(xtrain,ytrain)
evaluate_model(least_sqr,xtest,ytest)

R2 Score: 0.588
Mean Absolute Error normalized by the mean absolute value of y: 0.556
Mean Squared Error: 0.456


### Ridge Regression

Now a Ridge Regression. We perform a cross validated grid search in order to find the best weight $\alpha$.

In [95]:
param_grid_ridge = {'alpha': np.logspace(-4,3,128)}
grid_search_ridge = GridSearchCV(linear_model.Ridge(), param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_search_ridge.fit(xtrain, ytrain)
print("Best value for alpha: %1.3g" % grid_search_ridge.best_params_['alpha'])

ridge = grid_search_ridge.best_estimator_
ridge.fit(xtrain,ytrain)
evaluate_model(ridge,xtest,ytest)

Best value for alpha: 17.2
R2 Score: 0.592
Mean Absolute Error normalized by the mean absolute value of y: 0.555
Mean Squared Error: 0.451


### Lasso Regression

Finally, we perform Lasso Regression. We do the grid search.

In [96]:
param_grid_lasso = {'alpha': np.logspace(-4,2,128)}
grid_search_lasso = GridSearchCV(linear_model.Lasso(max_iter=50000,tol=0.001), param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
grid_search_lasso.fit(xtrain, ytrain)
print("Best value for alpha: %1.3g" % grid_search_lasso.best_params_['alpha'])

lasso = grid_search_lasso.best_estimator_
lasso.fit(xtrain,ytrain)
evaluate_model(lasso,xtest,ytest)

Best value for alpha: 0.00502
R2 Score: 0.587
Mean Absolute Error normalized by the mean absolute value of y: 0.557
Mean Squared Error: 0.457


## Conclusion

We compared different linear regression techniques. We found minor variations in the accuracy of the predictions when utilizing different schemes. Unfortunately, the predictions fluctuated a lot when changing the train/test split, which is done randomly. This leads me to believe that the linear regression models are not capable to properly predict the `hform` column given the feature space.